In [10]:
import os
import numpy as np 
import pandas as pd 
from subprocess import check_output
print(check_output(["ls", "./input"]).decode("utf8"))

4convlayers_0.1731.csv
4L_pseudolabeling_0.1763.csv
sub201801102212renset50_0.17478.csv
sub201801111028inceptionv3_0.1869.csv
sub201801111144_xception_0.1841.csv
sub201801111534ResNet50_aver_0.1689.csv
VGG16pretrain_0.1613.csv
VGG19_0.14.csv



In [11]:
sub_path = "./input"
all_files = os.listdir(sub_path)

# Read and concatenate submissions
outs = [pd.read_csv(os.path.join(sub_path, f), index_col=0) for f in all_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_5,is_iceberg_6,is_iceberg_7
0,5941774d,0.165616,0.040880,0.030369,0.003044,0.106100,1.483769e-05,1.758107e-01,0.035234
1,4023181e,0.440642,0.239020,0.364333,0.990582,0.302233,6.918396e-05,4.762523e-01,0.248752
2,b20200e4,0.157010,0.000118,0.017298,0.006188,0.000325,7.721067e-12,6.421022e-07,0.008674
3,e7f018bb,0.998759,0.948407,0.998199,0.999728,0.995213,9.999982e-01,9.970161e-01,0.988385
4,4371c8c3,0.199500,0.028982,0.657598,0.998925,0.045377,1.825918e-06,2.452419e-02,0.365645


In [4]:
# check correlation
concat_sub.corr()

,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_5,is_iceberg_6,is_iceberg_7
is_iceberg_0,1.000000,0.938561,0.622708,0.945130,0.813792,0.844431,0.908958,0.920000
is_iceberg_1,0.938561,1.000000,0.578125,0.978560,0.833962,0.873905,0.932555,0.937474
is_iceberg_2,0.622708,0.578125,1.000000,0.584820,0.573847,0.674491,0.610635,0.607147
is_iceberg_3,0.945130,0.978560,0.584820,1.000000,0.839452,0.868250,0.938131,0.935658
is_iceberg_4,0.813792,0.833962,0.573847,0.839452,1.000000,0.785209,0.820762,0.797363
is_iceberg_5,0.844431,0.873905,0.674491,0.868250,0.785209,1.000000,0.884527,0.859835
is_iceberg_6,0.908958,0.932555,0.610635,0.938131,0.820762,0.884527,1.000000,0.900926
is_iceberg_7,0.920000,0.937474,0.607147,0.935658,0.797363,0.859835,0.900926,1.000000


In [12]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:8].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:8].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:8].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:8].median(axis=1)

# MinMax + Median Stacking LB 0.1237

In [13]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.6578125
cutoff_hi = 0.3421875
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:8] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:8] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv('./sub4_with_0.14/stack_minmax_median_threshold_0.3421875_0.14model.csv', 
                                        index=False, float_format='%.6f')

In [ ]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.6578125
cutoff_hi = 0.3421875
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:8] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:8] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv('./sub4_with_0.14/stack_minmax_median_threshold_0.3421875_0.14model.csv', 
                                        index=False, float_format='%.6f')

# 10 MODEL ENSEMBLING

In [40]:
import os
import numpy as np 
import pandas as pd 
from subprocess import check_output
print(check_output(["ls", "./input"]).decode("utf8"))

0.1538.csv
4convlayers_0.1731.csv
4L_pseudolabeling_0.1763.csv
sub201801102212renset50_0.17478.csv
sub201801111028inceptionv3_0.1869.csv
sub201801111144_xception_0.1841.csv
sub201801111534ResNet50_aver_0.1689.csv
submission38.csv
VGG16pretrain_0.1613.csv
VGG19_0.14.csv



In [41]:
sub_path = "./input"
all_files = os.listdir(sub_path)

# Read and concatenate submissions
outs = [pd.read_csv(os.path.join(sub_path, f), index_col=0) for f in all_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_5,is_iceberg_6,is_iceberg_7,is_iceberg_8,is_iceberg_9
0,5941774d,0.023201,0.165616,0.040880,0.008875,0.030369,0.003044,0.106100,1.483769e-05,1.758107e-01,0.035234
1,4023181e,0.127432,0.440642,0.239020,0.146779,0.364333,0.990582,0.302233,6.918396e-05,4.762523e-01,0.248752
2,b20200e4,0.008776,0.157010,0.000118,0.048646,0.017298,0.006188,0.000325,7.721067e-12,6.421022e-07,0.008674
3,e7f018bb,0.994118,0.998759,0.948407,0.999184,0.998199,0.999728,0.995213,9.999982e-01,9.970161e-01,0.988385
4,4371c8c3,0.071480,0.199500,0.028982,0.277251,0.657598,0.998925,0.045377,1.825918e-06,2.452419e-02,0.365645


In [42]:
# check correlation
concat_sub.corr()

,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_5,is_iceberg_6,is_iceberg_7,is_iceberg_8,is_iceberg_9
is_iceberg_0,1.000000,0.921835,0.955681,0.991497,0.871976,0.581001,0.956529,0.855914,0.926055,0.912679
is_iceberg_1,0.921835,1.000000,0.938561,0.927501,0.844431,0.622708,0.945130,0.813792,0.908958,0.920000
is_iceberg_2,0.955681,0.938561,1.000000,0.956967,0.873905,0.578125,0.978560,0.833962,0.932555,0.937474
is_iceberg_3,0.991497,0.927501,0.956967,1.000000,0.878926,0.598208,0.958758,0.858070,0.928975,0.919062
is_iceberg_4,0.871976,0.844431,0.873905,0.878926,1.000000,0.674491,0.868250,0.785209,0.884527,0.859835
is_iceberg_5,0.581001,0.622708,0.578125,0.598208,0.674491,1.000000,0.584820,0.573847,0.610635,0.607147
is_iceberg_6,0.956529,0.945130,0.978560,0.958758,0.868250,0.584820,1.000000,0.839452,0.938131,0.935658
is_iceberg_7,0.855914,0.813792,0.833962,0.858070,0.785209,0.573847,0.839452,1.000000,0.820762,0.797363
is_iceberg_8,0.926055,0.908958,0.932555,0.928975,0.884527,0.610635,0.938131,0.820762,1.000000,0.900926
is_iceberg_9,0.912679,0.920000,0.937474,0.919062,0.859835,0.607147,0.935658,0.797363,0.900926,1.000000


In [43]:
N = 10

In [44]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:N].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:N].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:N].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:N].median(axis=1)

# LB0.1214

In [39]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.6578125
cutoff_hi = 0.3421875
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:N] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:N] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv('./sub4_with_0.14/stack_minmax_median_threshold_0.3421875_'+str(N)+'without0.1538model.csv', 
                                        index=False, float_format='%.6f')

# LB0.1214+

In [45]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.6574
cutoff_hi = 0.3426
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:N] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:N] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv('./sub4_with_0.14/stack_minmax_median_threshold_0.3426_'+str(N)+'model.csv', 
                                        index=False, float_format='%.6f')

In [47]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.6577
cutoff_hi = 0.3423
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:N] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:N] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv('./sub4_with_0.14/stack_minmax_median_threshold_0.3423_'+str(N)+'model.csv', 
                                        index=False, float_format='%.6f')

# 20180115 9model

In [1]:
import os
import numpy as np 
import pandas as pd 
from subprocess import check_output
print(check_output(["ls", "./input"]).decode("utf8"))

0.1538.csv
4convlayers_0.1731.csv
4L_pseudolabeling_0.1763.csv
sub201801111028inceptionv3_0.1869.csv
sub201801111144_xception_0.1841.csv
submission38.csv
VGG19_0.14.csv



In [33]:
sub_path = "./input"
all_files = os.listdir(sub_path)

# Read and concatenate submissions
outs = [pd.read_csv(os.path.join(sub_path, f), index_col=0) for f in all_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_5,is_iceberg_6
0,5941774d,0.023201,0.165616,0.008875,0.030369,0.003044,1.483769e-05,0.035234
1,4023181e,0.127432,0.440642,0.146779,0.364333,0.990582,6.918396e-05,0.248752
2,b20200e4,0.008776,0.157010,0.048646,0.017298,0.006188,7.721067e-12,0.008674
3,e7f018bb,0.994118,0.998759,0.999184,0.998199,0.999728,9.999982e-01,0.988385
4,4371c8c3,0.071480,0.199500,0.277251,0.657598,0.998925,1.825918e-06,0.365645


In [34]:
N = 7

In [35]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:N].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:N].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:N].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:N].median(axis=1)

In [40]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.6572125
cutoff_hi = 0.3427875
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:N] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:N] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv('./sub4_with_0.14/stack_minmax_median_threshold_'+str(cutoff_hi)+'_'+str(N)+'model.csv', 
                                        index=False, float_format='%.6f')

# push

In [2]:
import os
import numpy as np 
import pandas as pd 
from subprocess import check_output
print(check_output(["ls", "./sub4_with_0.14"]).decode("utf8"))

0.169.csv
stack_minmax_median_threshold_0.3421875_0.14model.csv
stack_minmax_median_threshold_0.3421875_10withoutXceptionmodel.csv
stack_minmax_median_threshold_0.3421875_11model_0.1219.csv
stack_minmax_median_threshold_0.3421875_6model.csv
stack_minmax_median_threshold_0.3421875_7model.csv
stack_minmax_median_threshold_0.3421875_8withoutR50M4Lmodel.csv
stack_minmax_median_threshold_0.3421875_9without0.1538model0.1217.csv
stack_minmax_median_threshold_0.3421875_9without38model0.1216.csv
stack_minmax_median_threshold_0.3421875_9withoutPseudomodel.csv
stack_minmax_median_threshold_0.3421875_9withoutResNet50Mmodel.csv
stack_minmax_median_threshold_0.3421875_EIGHTmodel.csv
stack_minmax_median_threshold_0.3421875_TENmodel_0.1214.csv
stack_minmax_median_threshold_0.3423_10model.csv
stack_minmax_median_threshold_0.3424_10model.csv
stack_minmax_median_threshold_0.3425875_7model.csv
stack_minmax_median_threshold_0.3426_10model.csv
stack_minmax_median_threshold_0.3427875_7model.csv
stack_minmax_

In [4]:
sub_path = './sub4_with_0.14'
a = sub_path + '/stack_minmax_median_threshold_0.3425875_7model.csv'
x = pd.read_csv(a)

In [33]:
print x[7000:]

            id  is_iceberg
7000  e50aea14    0.077621
7001  2cf3eb36    0.107681
7002  9e97e069    0.000000
7003  8981bb60    0.000000
7004  e6aa3bb1    0.172007
7005  841f7b98    0.353677
7006  2dd5583d    0.978069
7007  31ca7cf2    0.287126
7008  33551f78    0.000000
7009  ffef5f6b    0.999571
7010  e529b67b    0.115904
7011  0f047a04    0.474109
7012  6248644f    0.131592
7013  eff904a0    0.720677
7014  2a6f1a43    0.000000
7015  ab2cfec9    0.127880
7016  ddf0092f    0.999999
7017  de351321    0.276796
7018  25ae78a0    0.000000
7019  416a4b5e    0.406528
7020  7be7e4cb    0.000000
7021  70b2cc4e    0.000000
7022  62f23257    0.000000
7023  becb9167    0.135755
7024  248be600    0.261641
7025  8f94de6f    0.000000
7026  8a6019d6    0.308464
7027  7728fb89    0.195921
7028  d3318f72    0.000001
7029  d868e6e0    0.000000
...        ...         ...
8394  8ae30ce6    0.000000
8395  de27ed88    0.000003
8396  66d5196f    0.090725
8397  d85f1858    0.354729
8398  16dcb33a    0.509003
8

In [6]:
sub_path = "./bestsub"
all_files = os.listdir(sub_path)

In [7]:
print(check_output(["ls", sub_path]).decode("utf8"))

stack_minmax_median_threshold_0.3425875_7model.csv



In [8]:
# sub_path = "./input"
all_files = os.listdir(sub_path)

# Read and concatenate submissions
outs = [pd.read_csv(os.path.join(sub_path, f), index_col=0) for f in all_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,is_iceberg_0
0,5941774d,0.000015
1,4023181e,0.255556
2,b20200e4,0.000000
3,e7f018bb,0.999998
4,4371c8c3,0.238375


In [9]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:2].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:2].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:2].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:2].median(axis=1)

In [31]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.9645
cutoff_hi = 0.0355

In [32]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:2] > cutoff_lo, axis=1), 1, 
                                    np.where(np.all(concat_sub.iloc[:,1:2] < cutoff_hi, axis=1),
                                             0, concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv(sub_path + '/stack_pushout_median_'+str(cutoff_hi)+'.csv', 
                                        index=False, float_format='%.6f')

# 20180118 8model

In [1]:
import os
import numpy as np 
import pandas as pd 
from subprocess import check_output
print(check_output(["ls", "./input"]).decode("utf8"))

0.1538.csv
4convlayers_0.1731.csv
4L_pseudolabeling_0.1763.csv
LB1541_final_ensemble.csv
sub201801111028inceptionv3_0.1869.csv
sub201801111144_xception_0.1841.csv
submission38.csv
VGG19_0.14.csv



In [2]:
sub_path = "./input"
all_files = os.listdir(sub_path)

# Read and concatenate submissions
outs = [pd.read_csv(os.path.join(sub_path, f), index_col=0) for f in all_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3,is_iceberg_4,is_iceberg_5,is_iceberg_6,is_iceberg_7
0,5941774d,0.023201,0.165616,0.008875,0.030369,0.003044,1.483769e-05,0.006448,0.035234
1,4023181e,0.127432,0.440642,0.146779,0.364333,0.990582,6.918396e-05,0.758083,0.248752
2,b20200e4,0.008776,0.157010,0.048646,0.017298,0.006188,7.721067e-12,0.041054,0.008674
3,e7f018bb,0.994118,0.998759,0.999184,0.998199,0.999728,9.999982e-01,0.996372,0.988385
4,4371c8c3,0.071480,0.199500,0.277251,0.657598,0.998925,1.825918e-06,0.007940,0.365645


In [3]:
N = 8

In [4]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:N].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:N].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:N].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:N].median(axis=1)

In [5]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.6572125
cutoff_hi = 0.3427875
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:N] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:N] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv('./sub4_with_0.14/20180118stack_minmax_median_threshold_'+str(cutoff_hi)+'_'+str(N)+'model.csv', 
                                        index=False, float_format='%.6f')